# The script to visualise data from xiaomi scales (mifit app / zepp life app)
How to get the data: `open app` → `profile` → `settings` → `personal inf … privacy` → `Exercising user rights` → `Export data`  

----
how to prepare env  

```
poetry install
```
or 
```
pip install pandas matplotlib ipykernel seaborn
```

In [1]:
# !pip install pandas matplotlib ipykernel seaborn

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, AutoLocator

In [3]:
# Load the CSV file to inspect its structure and contents
file_path = "BODY_1736328346898.csv"

data = pd.read_csv(file_path)
data['time'] = pd.to_datetime(data['time'], utc=True)
data = data.sort_values(by='time')

# # Display the first few rows of the dataset and its column names
# data.head(), data.columns

In [4]:
# filter data
data = data[data['time'].dt.year >= 2024]
data = data[data['weight'] > 50] # keep only my weight (remove other people and pets)

# calculate values from rates
data['fat']      = data['weight'] * 0.01 * data['fatRate']
data['muscle']   = data['weight'] * 0.01 * data['muscleRate']
data['water']    = data['weight'] * 0.01 * data['bodyWaterRate']
data['visceral'] = data['weight'] * 0.01 * data['visceralFat']

# group data by day to ensure one point per day and fill missing dates with NaN
daily_data = data.set_index('time').resample('D').mean()
# daily_data.head()

In [5]:
# function to annotate points
def annotate_points(ax, x, y, n=10):
    min_idx = y.argmin()
    max_idx = y.argmax()
    special_indices = [0, len(y)-1, min_idx, max_idx]
    
    # Получаем значения сетки по оси Y
    yticks = ax.get_yticks()
    
    # Для каждой линии сетки находим ближайшее значение
    grid_matches = []
    for grid_value in yticks:
        # Находим разницу между всеми значениями и текущим значением сетки
        differences = abs(y - grid_value)
        # Находим индекс первого минимального значения
        first_closest_idx = differences.argmin()
        if abs(y[first_closest_idx] - grid_value) < 0.1:  # порог близости
            grid_matches.append(first_closest_idx)
    
    for idx, (date, value) in enumerate(zip(x, y)):
        # Специальные точки (мин, макс, первая, последняя)
        if idx in special_indices:
            if idx == min_idx:
                bbox_props = dict(boxstyle='round,pad=0.5', fc='lightgreen', alpha=0.7)
                xytext = (0, -25)
                va = 'top'
                ha = 'center'
            elif idx == max_idx:
                bbox_props = dict(boxstyle='round,pad=0.5', fc='lightcoral', alpha=0.7)
                xytext = (0, 20)
                va = 'bottom'
                ha = 'center'
            else:
                bbox_props = dict(boxstyle='round,pad=0.5', fc='white', alpha=0.7)
                if idx == 0:
                    xytext = (-15, 0)
                    ha = 'right'
                else:
                    xytext = (15, 0)
                    ha = 'left'
                va = 'center'
            
            ax.annotate(f'{value:.1f}', 
                       xy=(date, value),
                       xytext=xytext,
                       textcoords='offset points',
                       ha=ha,
                       va=va,
                       fontsize=10,
                       bbox=bbox_props,
                       arrowprops=dict(
                           arrowstyle='->',
                           connectionstyle='arc3,rad=0',
                           color='gray'
                       ))

In [ ]:
measurements = ["weight", "fat", "muscle", "water", "visceral", "bmi", "metabolism", "muscleRate"]

for measurement in measurements:
    # get min and max values for the current measurement
    value_min = daily_data[measurement].min()
    value_max = daily_data[measurement].max()

    # create a new figure with white background
    plt.figure(figsize=(10, 6), facecolor='white')
    ax = plt.gca()
    ax.set_facecolor('white')
    
    # plot scatter points without lines
    plt.plot(
        daily_data.index,
        daily_data[measurement],
        marker="o",
        markersize=2,
        linestyle='none'
    )
    
    # add padding to x-axis limits
    x_min, x_max = plt.xlim()
    x_range = x_max - x_min
    padding = x_range * 0.05
    plt.xlim(x_min - padding, x_max + padding)
    
    # set up major and minor grid lines
    ax.yaxis.set_major_locator(AutoLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))
    
    # add annotations to points
    annotate_points(plt.gca(), daily_data.index, daily_data[measurement], 16)
    
    # set up plot limits and labels
    plt.ylim(bottom=value_min * 0.95, top=value_max * 1.05)
    plt.title(f"{measurement}", fontsize=14)
    plt.xlabel("Date", fontsize=12)
    plt.ylabel(measurement.capitalize(), fontsize=12)
    
    # Configure grid
    plt.grid(True, which='major', color='gray', linestyle='-', alpha=0.2)
    plt.grid(True, which='minor', color='gray', linestyle='-', alpha=0.1)
    
    
    # add vertical line marking the first training date
    target_date = pd.to_datetime('2024-05-29', utc=True)
    plt.axvline(x=target_date, color='blue', linewidth=2, alpha=0.5)
    
    # add text label for the vertical line
    plt.text(target_date, value_min, '1st training session',
            rotation=90,          # vertical text
            va='bottom',          # align to bottom
            ha='right',           # align to right
            color='blue',         # same color as line
            alpha=0.5,            # same transparency as line
            fontsize=10,
            )

    
    # save figure to file
    plt.savefig(f'img_{measurement}.png', 
                dpi=300,               # high resolution
                bbox_inches='tight',   # trim empty edges
                facecolor='white',     # white background
                edgecolor='none')      # no edge color
    
    # 
    plt.show()
    plt.close()